In [8]:
# %%capture
!pip install snntorch tqdm matplotlib tensorboard

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 31.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 34.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 39.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ray 2.4.0 requires aiosignal, which is not installed.
ray 2.4.0 requires grpcio<=1.51.3,>=1.32.0; python_version < "3.10" and sys_platform != "darwin", but you have grpcio 1.66.2 which is incompatible.
ray 2.4.0 requires virtualenv<20.21.1,>=20.0.24, but you have virtualenv 20.24.5 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [3]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [1]:
import torch

# from src.Quantum_circuits import *
from src.QSVAE_model import *
from src.POVM_dataset import *

if torch.cuda.is_available():
 dev = "cuda:0"
#  dev = "cpu"
else:
 dev = "cpu"
device = torch.device(dev)
print(device)
torch.__version__

n = 3 # Amount of qubits
shots = 100_000 # amount of shots taken by the quantum simulator
first_run = False
# Support for "Starmon-5" and "AerSimulator" 
# backend_type = "AerSimulator"
# backend = select_backend(backend_type)
train = True
test = True
val = True

# Define hyperparameters
beta = 0.819
num_steps = 200
num_epochs = 1
learning_rate = 1e-3
batch_train, batch_test, batch_val = (10000, 200, 1000)
num_workers = 0
shuffle = False
split = [0.6, 0.2]



/srv/main-spack-instance-2402/spack/var/spack/environments/ebrains-24-04/.spack-env/view/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cpu


In [2]:
result, circuits, backend = None, None, None
# quantum_exp = QuantumExperiment(backend, n, shots)
# result, circuits = quantum_exp.run_experiment()

train_loader, test_loader, val_loader, POVM_dataset = load_data(result,
                                                                circuits,
                                                                first_run,
                                                                backend,
                                                                n,
                                                                shots,
                                                                split,
                                                                [batch_train, batch_test, batch_val],
                                                                shuffle, num_workers)

# Instantiate the model for the given parameters
model = SQVAE(n=n, batch_size=[batch_train, batch_train, batch_val],
              beta=beta, num_steps=num_steps, learning_rate=learning_rate,
                shots=shots, device=device, dataset=POVM_dataset)

# Run the model and get the fidelity for the current parameter setting
fidelity_score = model.run(train=True, test=False, val=True,
                           train_loader=train_loader, test_loader=test_loader,
                           val_loader=val_loader, num_epochs=num_epochs)




EOFError: Ran out of input